CELL 1!

In [4]:
%%time      
    
#CODE DESCRIPTION:
    #checking you're in the right conda environment / python version

#First, activate your conda environment via terminal "conda activate envrionment_name_here"
import sys 
print(sys.prefix) #checks what conda environment you're in


from platform import python_version # checks what python version you're in, should be "3.8.13"
print(python_version())


#CODE DESCRIPTION:
    #loads custom function file, re-run this code everytime you change & save the function file
    #imports all required modules
    
import Framework_functions_4_INTERVIEW_px
import imp 
imp.reload(Framework_functions_4_INTERVIEW_px) 
from Framework_functions_4_INTERVIEW_px import *

# import matlab.engine
 
from pylab import *

# import umap.umap_ as umap
import tensorflow
from tensorflow.keras.models import load_model

from sklearn.neighbors import KNeighborsClassifier

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, State, no_update, ctx
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
import dash_daq as daq
import dash_bootstrap_components as dbc
import copy
from skimage import measure
import scipy.io
from operator import itemgetter
import time
from datetime import datetime
import pickle
from IPython.display import clear_output



# from collections import OrderedDict

    #times of above cell: 2 ms

#CODE DESCRIPTION: 
    # in an effort to reduce latency of matlab-python connection, we open up matlab + kriging model workspace first (before entering interactive loop)
    # keep matlab open until interactive process is done, then eng.quit()
    # good to do this once only, otherwise everytime you predict compliance values, it takes an extra 1 min + of opening matlab/loading kriging model into workspac
    #some matlab engine info here:
        #https://www.mathworks.com/content/dam/mathworks/mathworks-dot-com/support/sysreq/files/python-compatibility.pdf
        #how to install matlab.engine: https://stackoverflow.com/questions/46141631/running-matlab-using-python-gives-no-module-named-matlab-engine-error
    #TASK: can this run faster?
    
#eng=matlab.engine.start_matlab()
#eng.Kriging_to_python_workspace(nargout=0)
eng = 0

# Load cGAN model
gmodel =  'cGANgen_cGANgc_vwh36_42000'
dmodel = 'cGANdisc_cGANgc_vwh36_42000'

project_dir = 'C:/software/ChrisCode/InteractiveFramework'
# project_dir = 'C:/Users/srv569/Documents/Interactive Framework Files/Run Interactive Framework - (Python + Matlab)'
gmodel_dir = project_dir + '/' + gmodel + '.h5'
dmodel_dir = project_dir + '/' + dmodel + '.h5'
gen_model = load_model(gmodel_dir, compile = False)
disc_model = load_model(dmodel_dir, compile =  False)
disc_model = tensorflow.keras.models.Model(inputs=disc_model.input,
                                  outputs=disc_model.layers[-2].output)
# --------------------------------------------------------------------------------------------------
# USER INPUT VARIABLES

mode = 'freeform'
r,c=100,10 # --> r x c images created
num_images = r*c
latent_dim = 12 # --> number of latent variables
cond_dim = 4
num_cond_inputs = 3
num_clusters = 4

# VARIABLES FOR PIXCLICK FUNCTION
num_variations = 12 # --> number of variations for pixclick
num_interp = 12 # --> number of interpolated images for pixclick
latent_range = 1.25 # for suggest & pixclick
latent_stretch = (0,1) # for pixclick
selected_area = (65, 85, 40, 60)
rect_color = (0.5, 0, 0)
rect_thickness = 2
# --------------------------------------------------------------------------------------------------

# CELL DESCRIPTION:
    # generating random conitional inputs &  random latent variables
    # using ^ to generate inital images w/ cGAN

# vfs = np.random.rand(num_images,1) # should be of shape (1,1)
# vfs = 0.4*vfs + 0.1 # note: should i normalize vf training data to 0-1 like sw & sh?
# sw = np.random.rand(num_images,1)
# sh = np.random.rand(num_images,1)

# vol_frac = np.repeat(vfs,cond_dim,axis=1) 
# width = np.repeat(sw,cond_dim,axis=1)
# height = np.repeat(sh,cond_dim,axis=1)
# cond_inputs = np.concatenate((vol_frac,width,height),axis=1)
# print(cond_inputs.shape)

# noise = np.random.rand(num_images,latent_dim)
# noise = 2.0*noise - 1 
# noise, gen_imgs =generate_imgs(num_images,noise,cond_inputs, gen_model)
# print("Check: images generated")

#     # times of above cell *UNSURE

# #CELL DESCRIPTION:
#     # predict comp scores
# comp_scores = gen_comp_vals(noise,eng)
# print("Check: compliance scores predicted")
    
# #CELL DESCRIPTION:
#     # cluster images based off visual similarity using kmeans 
    
# kpredictions=cluster_imgs(gen_imgs, num_clusters) 
# print("Check: images clustered")
    

#     #times of above cell *UNSURE
    
# #CELL DESCRIPTION:
#     # sort everything in order of clusters (e.g. 0 0 0 0....4 4 4 4)
#     # sort gen_imgs, noise, cond_inputs
# indices, kpred_sorted = zip(*sorted(enumerate(kpredictions), key=itemgetter(1)))
# noise = np.array([noise[i] for i in indices])
# gen_imgs = np.array([gen_imgs[i] for i in indices])
# cond_inputs = np.array([cond_inputs[i] for i in indices])
# ------------------------------------------------------------------------------------------

#CELL DESCRIPTION: 
    # creates a dictionary that will store all important information for each image
    # ^this dictionary should be the ONLY variable that's passed on to the interactive loop
    # ^ information stored in dict (for use in interactive loop):
        # each image's pixels, latent variables, compliance, color...
        # ..cluster #, image # w/in its cluster (e.g img #5 in cluster #2 is diff than img#5 in cluster#10)..
        # .. embed link (used to embed each image in the interactive plot's "hover " feature)
        # .. x,y points from UMAP projection


# px_data_array = np.array(gen_imgs, dtype=np.float64)
# nsamples, nx, ny, nz = px_data_array.shape
# px_data_final=px_data_array.reshape(nsamples, nx*ny*nz)

# main_dict = {}
# main_dict = {'cluster_num': list(kpred_sorted),
#              'compliance': [0],
#              'cond_inputs': cond_inputs,
#              'lv_data': noise,
#              'px_data': px_data_array,
#              'px_avg_data': px_avg_data_array} # avg img edit

# print("Check: main_dict initiated")
#     #times of above cell: 799 ms, 850 ms

# main_dict['dims'] = []
# main_dict['cost'] = []
# for i in range(num_images):
#     lca = frameworkCalcs(all_data['px_data'][i,:,:,0])
#     main_dict['cost'].append(lca[-1])

# ------------------------------------------
# EXPORT MAIN DICT TO EXTERNAL FILE (SEED)
# with open('main_dict.pickle','wb') as f:
#     pickle.dump(main_dict,f)

C:\software\.conda\envs\framework_env
3.8.13
CPU times: total: 5.84 s
Wall time: 5.74 s


CELL 2!

In [5]:
%%time

# ----------------------------------------------------------------------------------
# option 1) deep copy of main_dict to make edits
# all_data = copy.deepcopy(main_dict)
# Clear the output of the current cell


# option 2) LOADING SEED
with open('freeformseed_withcost.pickle', 'rb') as f:
    all_data = pickle.load(f)
    
all_data['cost'] = []
for i in range(num_images):
    lca = frameworkCalcs(all_data['px_data'][i,:,:,0])
    all_data['cost'].append(lca[-1])
# ---------------------------------------------------------------------------------

# Store Usage Data (for user study)
usage_log = []
current_tab = "1"
tab_start = time.time()
tab_end = 0

# User Study Data Logs
sets_explored = []
# trying out attaching function count to usage log
function_count = [0,0]

# ADD CODE SNIPPET TO ADD ALL_DATA['COST'] THEN EXPORT THE SEED AGAIN


# variables for generating images
gen_model = load_model(gmodel_dir)

# Global variables for storing data
# resets all values to [] so previous run doesn't carry over
# NOTE: all the 'current' global variables could be consolidated into one current_dict (same with others - suggest/interp/etc)
first_click = False
current_cost = 0
current_lv_data = []
current_cond_data = []
current_dimensions = []
saved_imgs=[-1]
img_data = []
int_noise = []
int_gen_imgs = []
int_comp_scores = []
interp_imgs = []
interp_noise = []
interp_bcs = []
suggest_img_px_data = []
suggest_img_lv_data = []
suggest_img_cond_data = []
suggest_img_dimensions = []
sidebar_len = 10
saved_imgs_source = []


# first & last indices of each cluster
first_indices = [0]
last_indices = []
for i in range(1,4):
    next_index = all_data['cluster_num'].index(i)
    first_indices.append(next_index)
    last_indices.append(next_index-1)
last_indices.append(r*c-1)

# CHRIS EDIT
# SAME AS ABOVE BUT FOR AVERAGE IMAGES / CLUSTER # avg img edit
graph_style = {'max-height':500}
avg_graphs = []
for i in range(num_clusters):
    # creating dcc.graph
    avg_image_data = 255-(all_data['px_avg_data'][i,:,:,0]*255) # 128x96
    avg_graph = dcc.Graph(figure=make_figure(avg_image_data,300))
    avg_graphs.append(avg_graph)


# keeping track of 8 images displayed
set_count = [(1+(last_indices[i]-first_indices[i])//8) for i in range(4)]
current_set = [0,0,0,0] # for each cluster
# print(set_count)

#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_stylesheets=[dbc.themes.BOOTSTRAP]
app = JupyterDash(external_stylesheets=external_stylesheets)

server = app.server

# -----------------------------------------------------------------------------------
# DATA NEEDED FOR TABS

# CREATE LIST OF DCC.GRAPHS + COMPUTE HEIGHT/WIDTH (PIXEL-WISE)
graph_style = {'max-height':500}
graphs = []
for i in range(r*c):
    # creating dcc.graph
    image_data = 255-(all_data['px_data'][i,:,:,0]*255) # 128x96
    graph = dcc.Graph(figure=make_figure(image_data,300))
    graphs.append(graph)
    
    # activate when error is solved
    dim_list = compute_dimensions(image_data)
    all_data["dims"].append(dim_list)

# STYLES
nav_button_style = {'width':'75%','height':'50%','font-weight':'bold'}
nav_button_pos = {'width':'25%','display':'flex','justify-content':'center','align-items':'center'}
nav_bar_style = {'z-index':'100','display':'flex','width':'100%','background-color':'#A4FED6','height':'10vh','position':'fixed','bottom':'0'}

sidebar_fig_style = {'width':'100%','display': 'inline-block', 'margin': '10px','cursor':'pointer'}
sidebar_style = {'overflowY':'scroll','width': '30%', 'height': '90vh', 'float': 'left', 'margin-top':'5vh'}

tab2_row1_style = {'height':'40vh','width':'70%','margin-top':'5vh','float':'right','whiteSpace':'nowrap','overflow':'visible','padding':'10px'}
tab2_row2_style = {'height':'40vh','width':'70%','float':'right','whiteSpace':'nowrap','overflow':'visible','padding':'10px'}
tab2_fig_style = {'display': 'inline-block','width':'25%', 'height': '20vh'}
fig_label_style = {'width':'100%','text-align':'center','align-items':'center'}

# -----------------------------------------------------------------------------------
# ALL TAB CONTENTS (HTML STUFF)

# Setting up sidebars

blank_image = np.zeros((128, 128, 3), dtype=np.uint8)
sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])

feedback_style = {'z-index':'100','width':'100%','background-color':'#A4FED6','height':'5vh','position':'fixed','top':'0','text-align':'center','align-items':'center','font-weight':'bold'}

sidebar_components = {}
for i in ["2c0","2c1","2c2","2c3","3","4","lv","5","6","fin"]: # sv edit 2/20
    sidebar_component = [
        html.Div(id=f'{i}-container-{j}', n_clicks=0, children=[
            html.Div(children=[
                    html.P(f'Saved Image {j+1}: Empty', id=f'{i}-label-{j}', style={'width':'50%','color':'red','font-weight':'bold'}),
                    html.Button('Delete', id=f'{i}-delete-{j}',n_clicks=0, style={'width':'50%'})
                ], style={'display':'flex'}
            ),
            dcc.Graph(
                id=f'{i}-saved-image-{j}',
                figure={
                    'data': [{'type': 'image', 'z': blank_image}],
                    'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                },
                style=sidebar_fig_style
            )
        ])
        for j in range(sidebar_len)
    ]
    sidebar_components[i] = sidebar_component # add sidebar as dictionary entry

# Create html layout for each sidebar
sidebar_list = [html.Div([
            html.Div(id=f'sidebar-{i}',
                     children=sidebar_components[i])
        ], style=sidebar_style)
    for i in ["2c0","2c1","2c2","2c3","3","4","lv","5","6","fin"] # sv edit 3/20
]


# Setting up Dropdowns to make code cleaner
dropdown_options = update_dropdown_options(saved_imgs)

blend_dropdown_1 = html.Div([
        html.Label("Choose Image 1", htmlFor="blend-dropdown-1"),
        dcc.Dropdown(id='blend-dropdown-1',
                    options=dropdown_options,
#                     value=saved_imgs[0]
        )
    ],
    style={'width':'50%'}
)
blend_dropdown_2 = html.Div([
        html.Label("Choose Image 2", htmlFor="blend-dropdown-2"),
        dcc.Dropdown(id='blend-dropdown-2',
                    options=dropdown_options,
#                     value=saved_imgs[0]
        )
    ],
    style={'width':'50%'}
)

# Setting up Floating control buttons (4xbuttons)
# REVISION: ALL TABS HAVE SAME BUTTON LAYOUT (4 BUTTONS)
float_buttons = [
    html.Div([
        html.Div(
            html.Button("Back to Styles", id=f'{j}-to-1', n_clicks=0, style=nav_button_style),
            style=nav_button_pos
        ),
        html.Div(
            html.Button("Edit a Saved Design", id=f'{j}-to-inter', n_clicks=0, style=nav_button_style),
            style=nav_button_pos
        ),
        html.Div(
            html.Button("3D View a Saved Design", id=f'{j}-to-3', n_clicks=0, style=nav_button_style),
            style=nav_button_pos
        ),
        html.Div(
            html.Button("Print a Saved Design", id=f'{j}-to-fin', n_clicks=0, style=nav_button_style),
            style=nav_button_pos
        ),
    ],style=nav_bar_style)
    for j in ["1","2c0","2c1","2c2","2c3","3","inter","4","lv","5","6","fin"] # sv edit 1/20
]



# CREATE ALL TAB 2 HTML LAYOUTS
tab_2_list = [dcc.Tab(label=f'Cluster {j+1}', value=f'tab-2c{j}', children=[
        html.H4("Click on designs you like to save to the sidebar", id=f'2c{j}-feedback', style=feedback_style),
        # Display all images in cluster flexibly
        # TESTING EDIT: SHOW 2X4 GRID OF IMAGES IN EACH CLUSTER PAGE
        html.Div(
#             id=f'cluster-display-{j}',
            children=[
                html.Div(id=f'c{j}-slot{i}',children=[
                    html.H6(f'Cluster {j} Image {i}: ${all_data["cost"][first_indices[j]+i]}',
                            id=f'c{j}-image{i}-title',
                            style={'color':'red',"font-weight":'bold'}), # CHRIS EDIT
#                     graphs[i]
                    # putting the id's under dcc.Graphs to access in callbacks
                    dcc.Graph(
                        id=f'c{j}-image{i}',
                        figure = make_figure(255-(all_data['px_data'][i+first_indices[j],:,:,0]*255), 300)
                    )
                ], 
                n_clicks=0, style=tab2_fig_style)
                for i in range(4)
            ],
            style=tab2_row1_style
        ),
        sidebar_list[j],
        html.Div(
#             id=f'cluster-display-{j}',
            children=[
                html.Div(id=f'c{j}-slot{i}',children=[ # slot numbers are from 0~7
                    html.H6(f'Cluster {j} Image {i}: ${all_data["cost"][first_indices[j]+i]}',
                            id=f'c{j}-image{i}-title',
                            style={'color':'red',"font-weight":'bold'}), # CHRIS EDIT
                    dcc.Graph(
                        id=f'c{j}-image{i}', # graph numbers are also from 0~7
                        figure = make_figure(255-(all_data['px_data'][i+first_indices[j],:,:,0]*255), 300),
                        style = {'height':300}
                    )
                ], 
                n_clicks=0, style=tab2_fig_style)
                for i in range(4,8)
            ],
            style=tab2_row2_style
        ),
        html.Div(
            children=[
                html.Button('<----',id=f'c{j}-left',n_clicks=0,style={'width':'200px','margin-left':'100px'}),
                html.Button('---->',id=f'c{j}-right',n_clicks=0,style={'width':'200px','margin-right':'100px'})
            ],
            style={'display':'flex','justify-content':'space-between','height':'5vh','width':'70%','float':'right','margin-bottom':'10vh'} 
        ),
        
        float_buttons[j+1]
    ]) for j in range(4)
]

# ---------------------------------------------------
# ----------------------ALL TABS LAYOUT------------------
# ---------------------------------------------------

app.layout = html.Div([
    dcc.Tabs(id="tabs-example", value='tab-1', 
             style={'display': 'none'}, # hides tab ribbon on top
             children=[
        # ---------TAB1 : CHOOSE CLUSTER
        dcc.Tab(label='Style', value='tab-1', children=[
            html.Div([
                html.H4('Click on an Image to explore more similar designs! You can come back to this page.', id='1-feedback', style=feedback_style),
                html.Div([
                    html.Div([
#                             html.H6("Cluster 1",style=fig_label_style),
                            avg_graphs[0] # CHRIS EDITS
                        ], 
                        style={'width':'50%','height':'90%'}, id='cluster_0_image', n_clicks=0
                    ),
                    html.Div([
#                             html.H6("Cluster 2",style=fig_label_style),
                            avg_graphs[1]
                        ], 
                        style={'width':'50%','height':'90%'}, id='cluster_1_image', n_clicks=0
                    )],
                    style={'display':'flex','height':'42vh'}
                ),
                html.Div([
                    html.Div([
#                             html.H6("Cluster 3",style=fig_label_style),
                            avg_graphs[2]
                        ], 
                        style={'width':'50%','height':'90%'}, id='cluster_2_image', n_clicks=0
                    ),
                    html.Div([
#                             html.H6("Cluster 4",style=fig_label_style),
                            avg_graphs[3]
                        ], 
                        style={'width':'50%','height':'90%'}, id='cluster_3_image', n_clicks=0
                    )],
                    style={'display':'flex','margin-bottom':'10vh','height':'42vh'}
                )],
                style={'overflowY':'scroll','margin-top':'5vh'}
            ),
            float_buttons[0],
        ]),
        # ------------TAB2 : DISPLAY EACH CLUSTER
        tab_2_list[0],
        tab_2_list[1],
        tab_2_list[2],
        tab_2_list[3],
        # ------------TAB3 : 3D RENDERING
        dcc.Tab(label='3D View', value='tab-3', children=[
            html.H4('Choose a saved image to view in 3D.', id='3-feedback', style=feedback_style),
            html.Div([
                dcc.Dropdown(id='3d-dropdown',
                    options= dropdown_options,
                    value=saved_imgs[0]),
                html.Div([
                    html.Button("Extruded",id='extrude-button',n_clicks=0,style={'width':'20%','height':'100%','margin':'0 15% 0 15%'}),
                    html.Button("Revolved",id='revolve-button',n_clicks=0,style={'width':'20%','height':'100%','margin':'0 15% 0 15%'}),
                ], style={'height':'5vh','align-items':'center','margin':'1vh 0 1vh 0'}
                ),
                dcc.Graph(
                    id='3d-render',
                    figure={
                        'data': [{'type': 'image', 'z': blank_image}],
                        'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                    }
                )],
                style={'width':'70%','float':'right','margin-top':'5vh'}
            ),
            sidebar_list[4],
            float_buttons[5],
        ]),
        # ------------INTERMEDIATE TAB : NAVIGATE TO ADJUST OR BLEND
        dcc.Tab(label='Intermediate',value='tab-inter',children=[
            html.Div(children=[
                html.H4("Choose an Editing Option"),
                html.Div(
                    html.Button("Edit a Chair!",
                                id='inter-to-4',
                                n_clicks=0,
                                style={'width':'300px','height':'100px'}
                    ),
                    style={'display':'flex','justify-content':'center','align-items':'center','height':'20vh'}
                ),
                html.Div(
                    html.Button("Blend 2 Chairs Together!",
                                id='inter-to-5',
                                n_clicks=0,
                                style={'width':'300px','height':'100px'}
                    ),
                    style={'display':'flex','justify-content':'center','align-items':'center','height':'20vh'}
                ),
                html.Div(
                    html.Button("Try Random Changes",
                                id='inter-to-lv',
                                n_clicks=0,
                                style={'width':'300px','height':'100px'}
                    ),
                    style={'display':'flex','justify-content':'center','align-items':'center','height':'20vh'}
                ),
                html.Div( # sv edit 4/20: inter --> suggest tab button
                    html.Button("Get Design Suggestions",
                                id='inter-to-6',
                                n_clicks=0,
                                style={'width':'300px','height':'100px'}
                    ),
                    style={'display':'flex','justify-content':'center','align-items':'center','height':'20vh'}
                ),
                float_buttons[6],
            ])
        ]),
        # ------------TAB4 : COND.INPUTS AND PIXCLICK ADJUSTMENT
        dcc.Tab(label='Adjust', value='tab-4', children=[
            html.H4('Choose a saved design to customize.', id='4-feedback', style=feedback_style),
            html.Div([
                # Interface Screen
                html.Div([
    #                     html.P("Choose image to modify:"),
                        dcc.Dropdown(id='adjust-dropdown',
                                     options=dropdown_options,
                                    value=saved_imgs[0]),
                        html.P("You can drag a rectangle on the image to edit a specific area",id='pixclick-feedback'),
                        dcc.Graph(
                            id='adjust-image',
                            figure={
                                'data': [{'type': 'image', 'z': blank_image}],
                                'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                            }
                        ),
                        html.Div(
                            html.Button("Save as new image", id='adjust-save-button', n_clicks=0, style={'width':'200px'}),
                            style={'width':'100%','justify-content':'center','align-items':'center'}
                        ),
    #                     html.Pre("(default)",id="pixclick-feedback",style={'margin-bottom':'5px'}),
                        html.P(
                            f"Height: \nLeg Width: \nSeat Width:",
                            id="adjust-dim",
                            style={"white-space":'pre-line'}
                        ),
                        html.P(
#                             f"By creating this design, save<>",
                            f"Printing this design costs $<>",
                            id="adjust-lca",
                            style={"white-space":'pre-line','color':'red',"font-weight":'bold'} # CHRIS EDIT
                        ),
                    ],
                    style={'width':'60%','margin-right':'10%','margin-bottom':'10vh'}
                ),
                # -----------------------------------------------------------------------------
                # PIXCLICK INTEGRATION --------------------------------------------------------
                # -----------------------------------------------------------------------------
                # Sliders
                html.Div([
                        html.P('Thickness',style={'margin-bottom':'40px'}),
                        daq.Slider(min=0.1, max=0.5, step=0.05,
                                id='lv_vf', updatemode='drag',
                                value=0, handleLabel={'showCurrentValue':True, "label":"VALUE"},
                        ),
                        html.P('Width',style={'margin-top':'10px','margin-bottom':'40px'}),
                        daq.Slider(min=0, max=1, step=0.1,
                                id='lv_sw', updatemode='drag',
                                value=0, handleLabel={'showCurrentValue':True, "label":"VALUE"}
                        ),
                        html.P('Height',style={'margin-top':'10px','margin-bottom':'40px'}),
                        daq.Slider(min=0, max=1, step=0.05,
                                id='lv_sh', updatemode='drag',
                                value=0, handleLabel={'showCurrentValue':True, "label":"VALUE"}
                        ),
                        html.P('Selected Area: Drag a Rectangle on the Image First',style={'margin-top':'10px','margin-bottom':'40px'}),
                        daq.Slider(min=0, max=num_interp-1, step=1,
                                  id='interp-slider', value=0, updatemode='drag'),
                        html.Button('Try Random Edits',id='4-to-lv',n_clicks=0,style={'margin-top':'40px', 'color': 'red'}),
                        html.Button('Get Suggestions',id='4-to-6',n_clicks=0,style={'margin-top':'40px','color': 'red'}),
                        
                        html.Div(style={'margin-bottom':'10vh'})
                    ],
                    style={'width':'30%'}
                )],
                style={'width':'70%','float':'right','display':'flex','margin-top':'5vh'}
            ),
            sidebar_list[5],
            float_buttons[7]
        ]),
        # -----------TAB 4-2: LV SLIDERS
        dcc.Tab(label='Random Edits', value='tab-4-2', children=[
            html.H4('Try out random changes by moving the sliders.', id='lv-feedback', style=feedback_style),
            html.Div([
                # Interface Screen
                html.Div([
                        html.P("Choose image to modify:"),
                        dcc.Dropdown(id='lv-dropdown',
                                     options=dropdown_options,
                                    value=saved_imgs[0]),
                        html.H3(" "),
                        dcc.Graph(
                            id='lv-image',
                            figure={
                                'data': [{'type': 'image', 'z': blank_image}],
                                'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                            }
                        ),
                        html.Div(children=[
                                html.Div(
                                    html.Button("Save as new image", id='lv-save-button', n_clicks=0, style={'width':'200px'}),
                                    style={'width':'50%','align':'center'}
                                ),
                                html.Div(
                                    html.Button("Back to Edit Page", id='lv-to-4',n_clicks=0, style={'width':'200px'}),
                                    style={'width':'50%','align':'center'}
                                ),
                            ], style={'display':'flex','align-items':'center','justify-content':'center'}
                        ),

                        html.P(
                            f"Height: \nLeg Width: \nSeat Width:",
                            id="lv-dim",
                            style={"white-space":'pre-line'}
                        ),
                        html.P(
#                             f"By creating this design, save<>",
                            f"Printing this design costs $<>",
                            id="lv-lca",
                            style={"white-space":'pre-line','color':'red',"font-weight":'bold'} # CHRIS EDIT
                        ),
                    ],
                    style={'width':'60%','margin-right':'5%'}
                ),
                # LV Sliders
                html.Div([
                        html.Div(children=
                                [html.Div([
                                        html.P(f'Random Edit #{i}',style={'margin-top':'10px','margin-bottom':'40px'}),
                                        daq.Slider(min=-1, max=1, step=0.2,
                                            id=f'lv_{i}', updatemode='drag',
                                            value=0,
                                            handleLabel={'showCurrentValue':True, "label":"VALUE"}
                                        ),
                                ]) for i in range(1,latent_dim+1)]
                        ),
#                         html.Div(style={'margin-bottom':'10vh'})
                    ],
                    style={'width':'30%','height':'85vh','overflowY':'hidden'} # CHRIS EDITS
                )],
                style={'width':'70%','float':'right','display':'flex','margin-top':'5vh'}
            ),
            sidebar_list[6],
            float_buttons[8]
        ]),
        # ------------TAB5 : BLEND
        dcc.Tab(label='Blend', value='tab-5', children=[
            html.H4("Choose two saved images to interpolate between", id='5-feedback', style=feedback_style),
            html.Div([
                    html.Div(children=[
                            blend_dropdown_1,
                            blend_dropdown_2,
                        ],
                        style=dict(display='flex')
                    ),
                    html.Div(
                        dcc.Graph(
                            id='blended-image',
                            figure={
                                'data': [{'type': 'image', 'z': blank_image}],
                                'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                            }
                        )
                    ),
                    html.Div(children=[
                            html.Div(
                                daq.Slider(min=0, max=9, step=1,
                                    id='blender',
                                    value=0, updatemode='drag',
                                    handleLabel={'showCurrentValue':True, "label":"VALUE"}
                                ),
                                style={'width':'40%','align':'center','margin-left':'30%'}
                            ),
                            html.Div(
                                html.Button("Save as new image", id='blend-save-button', n_clicks=0, style={'width':'200px'}),
                                style={'width':'30%','align':'right'}
                            ),
                        ], style={'display':'flex'}
                    ),
                    html.P(
                        f"Height: \nLeg Width: \nSeat Width:",
                        id="blend-dim",
                        style={"white-space":'pre-line'}
                    ),
                    html.P(
#                         f"By creating this design, save<>",
                        f"Printing this design costs $<>",
                        id="blend-lca",
                        style={"white-space":'pre-line','color':'red',"font-weight":'bold'} # CHRIS EDIT
                    ),
                ],
                style={'width':'70%','float':'right','margin-top':'5vh'}
            ),
            sidebar_list[7],
            float_buttons[9]
        ]),
        
         # ---------------TAB 6: DESIGN SUGGESTIONS  # sv edit 5/20: new tab

        dcc.Tab(label='Suggest', value='tab-6', 
                children = [html.H4("Choose a saved design, then click the Suggest Designs button", id='6-feedback', style=feedback_style),
                            html.Div([
                                     dcc.Dropdown(id='suggest-dropdown',
                                             options=dropdown_options,value=saved_imgs[0], style={'width':'50%','margin-bottom':'3vh'}),
                                     html.Div(children=[
                                             html.Button('Generate Design Suggestions', id='suggest-button', n_clicks=0, style={'width':'40%'}),
                                             html.Button('Back to Edit Page', id='6-to-4',n_clicks=0, style={'width':'40%'})
                                         ],
                                         style={'width':'60%','display':'flex','margin-bottom':'3vh'}
                                     ),
                                     
                                     html.Div(id=f'suggest-display-1',
                                              children = [html.Div([html.H6('Image 1', id='suggest-title-1', style={'color':'red',"font-weight":'bold'}),
                                                                    dcc.Graph(id='suggest-image-1',figure={
                                                                        'data': [{'type': 'image', 'z': blank_image}],
                                                                        'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                                                                    }),
                                                                    html.Button("Save",id='suggest-save-1',n_clicks=0,style={'height':'5vh','width':'200px'})
                                                                   ], id='suggestion-1', n_clicks=0, style=tab2_fig_style),
                                                          html.Div([html.H6('Image 2', id='suggest-title-2', style={'color':'red',"font-weight":'bold'}),
                                                                    dcc.Graph(id='suggest-image-2',figure={
                                                                        'data': [{'type': 'image', 'z': blank_image}],
                                                                        'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                                                                    }),
                                                                    html.Button("Save",id='suggest-save-2',n_clicks=0,style={'height':'5vh','width':'200px'})
                                                                   ], id='suggestion-2', n_clicks=0, style=tab2_fig_style)
                                              ],
                                     ),

                            ],style={'height': '85vh','width':'70%','float':'right','whiteSpace': 'nowrap','padding': '10px','margin-bottom':'10vh','margin-top':'5vh'}),
                            sidebar_list[8],
                            float_buttons[10]
                ]
        ),
        
        # FINISH TAB
        dcc.Tab(label='Finish', value='tab-fin', children=[
            html.H4("Make your final choice!",id='fin-feedback',style=feedback_style),
            html.Div([
                dcc.Dropdown(id='fin-dropdown',
                    options=dropdown_options,
                    value=saved_imgs[0]),
                html.Div([
                        html.Button("Extruded",id='extrude-button-fin',n_clicks=0,style={'width':'20%','height':'100%','margin':'0 15% 0 15%'}),
                        html.Button("Revolved",id='revolve-button-fin',n_clicks=0,style={'width':'20%','height':'100%','margin':'0 15% 0 15%'}),
                    ], style={'height':'5vh','align-items':'center','margin-top':'1vh'}
                ),
                dcc.Graph(
                    id='fin-render',
                    figure={
                        'data': [{'type': 'image', 'z': blank_image}],
                        'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                    }
                ),
                html.Div(children=[
                    html.Button("Final Choice-EXTRUDED", id='fin-button-ext', n_clicks=0, style={'width':'30%','height':'100%','margin':'0 10% 0 10%'}),
                    html.Button("Final Choice-REVOLVED", id='fin-button-rev', n_clicks=0, style={'width':'30%','height':'100%','margin':'0 10% 0 10%'})],
                    style={'height':'5vh','align-items':'center','margin-bottom':'10vh'}
                )],
                style={'width':'70%','float':'right','margin-top':'5vh'}
            ),
            sidebar_list[9],
            float_buttons[11]
        ])
    ])
])

# -----------------------------------------------------------------------------------
# -------------------------------CALLBACKS BEGIN HERE--------------------------------
# -----------------------------------------------------------------------------------


# ************TAB 1 CALLBACKS***************
# floating buttons control
@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'{i}-to-1','n_clicks') for i in ["2c0","2c1","2c2","2c3","3","inter","4","lv","5","6","fin"]],
    prevent_initial_call=True
)
def display_tab_1(*args):
    print('\n*************************callback: display_tab_1')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)


    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "1"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-1'


# ************TAB 2 CALLBACKS***************


# TAB 2: OPEN TAB
@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'cluster_{i}_image', 'n_clicks') for i in range(4)],
    prevent_initial_call=True
)
def select_cluster(*args):
    print("\n*************************callback: select_cluster")
    cluster_index = int(ctx.triggered_id.split('_')[1]) # extract selected cluster#

    global tab_start, tab_end, current_tab, usage_log, sets_explored, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)
    function_count = [0,0]

    # reset the data, so the next callback can record the time spent on this tab
    current_tab = f"C{cluster_index}"
    tab_start = time.time() # start measuring time spent (for next tab)

    set_str = f"c{cluster_index}_set0"
    if set_str not in sets_explored:
        sets_explored.append(set_str)

    return f"tab-2c{cluster_index}"

# TAB 2: UPDATE SIDEBARS AFTER OPENING TAB, SAVING AN IMAGE, OR DELETING AN IMAGE
@app.callback(
    [Output(f'2c0-saved-image-{j}', 'figure', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f'2c0-label-{j}', 'children', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f"2c0-feedback","children",allow_duplicate=True)],
    Input('cluster_0_image', 'n_clicks'),
    [Input(f'c0-slot{a}', 'n_clicks') for a in range(8)],
    [Input(f"2c0-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    prevent_initial_call=True
)
def update_sidebar_c0(*args):
    print("\n*************************callback: update_sidebar_0")
    global saved_imgs, saved_imgs_source, sidebar_outputs
   
    triggered_id = ctx.triggered_id
    print(f'trigger: {triggered_id}')
    if 'cluster' in triggered_id:
        feedback_msg = 'Click on a design to save it'
    elif 'slot' in triggered_id:
        # Handle save_clicked_img_c3 trigger
        cluster = int(ctx.triggered_id[1])
        slot_no = int(ctx.triggered_id.split('slot')[-1])
        set_no = current_set[cluster]
        saved_img_no = first_indices[cluster] + set_no * 8 + slot_no
        
        saved_imgs, feedback_msg = update_saved_imgs(saved_img_no, saved_imgs, sidebar_len)
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
        saved_imgs_source.insert(0, "c0")
        
        feedback_msg = f'Design has been saved'
    elif 'delete' in triggered_id:
        # Handle delete_saved_img trigger
        img_to_del = int(triggered_id.split('-')[-1])
        print(f'saved_imgs before: {saved_imgs}')
        print(f'img_to_del:{img_to_del}')
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            print(f'saved_imgs after: {saved_imgs}')
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg = f'Design has has been deleted'
        else:
            feedback_msg = 'There is no design to delete'
            
    return sidebar_outputs + [feedback_msg]

        
@app.callback(
   
    [Output(f'2c1-saved-image-{j}', 'figure', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f'2c1-label-{j}', 'children', allow_duplicate=True) for j in range(sidebar_len)],
    [Output('2c1-feedback','children',allow_duplicate=True)],
    
    Input('cluster_1_image', 'n_clicks'),
    [Input(f'c1-slot{a}', 'n_clicks') for a in range(8)],
    [Input(f"2c1-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    prevent_initial_call=True
)
def update_sidebar_c1(*args):
    print("\n*************************callback: update_sidebar_c1")
    global saved_imgs, sidebar_outputs
    triggered_id = ctx.triggered_id
    print(f'trigger: {triggered_id}')
    if 'cluster' in triggered_id:
        feedback_msg = "Click a design to save it"
    elif 'slot' in triggered_id:
        # Handle save_clicked_img_c3 trigger
        cluster = int(ctx.triggered_id[1])
        slot_no = int(ctx.triggered_id.split('slot')[-1])
        set_no = current_set[cluster]
        saved_img_no = first_indices[cluster] + set_no * 8 + slot_no
        
        saved_imgs, feedback_msg = update_saved_imgs(saved_img_no, saved_imgs, sidebar_len)
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
        saved_imgs_source.insert(0, "c1")
        
        feedback_msg = f"Design has been saved"
    elif 'delete' in triggered_id:
        # Handle delete_saved_img trigger
        img_to_del = int(triggered_id.split('-')[-1])
       
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg = f"Design has has been deleted"
        
    return sidebar_outputs+ [feedback_msg]  
        
        
@app.callback(
    [Output(f'2c2-saved-image-{j}', 'figure', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f'2c2-label-{j}', 'children', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f"2c2-feedback","children",allow_duplicate=True)],
    Input('cluster_2_image', 'n_clicks'),
    [Input(f'c2-slot{a}', 'n_clicks') for a in range(8)],
    [Input(f"2c2-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    prevent_initial_call=True
)
def update_sidebar_c2(*args):
    print("\n*************************callback: update_sidebar_c2")
    global saved_imgs, sidebar_outputs
   
    triggered_id = ctx.triggered_id
    print(f'trigger: {triggered_id}')
    if 'cluster' in triggered_id:
        feedback_msg = 'Click a design to save it'

    elif 'slot' in triggered_id:
        # Handle save_clicked_img_c3 trigger
        cluster = int(ctx.triggered_id[1])
        slot_no = int(ctx.triggered_id.split('slot')[-1])
        set_no = current_set[cluster]
        saved_img_no = first_indices[cluster] + set_no * 8 + slot_no
        
        saved_imgs, feedback_msg = update_saved_imgs(saved_img_no, saved_imgs, sidebar_len)
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
        saved_imgs_source.insert(0, "c2")
        
        feedback_msg = f'Design has been saved '
    elif 'delete' in triggered_id:
        # Handle delete_saved_img trigger
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
        feedback_msg = f'Design has been deleted'
    return sidebar_outputs + [feedback_msg]
        
@app.callback(
    [Output(f'2c3-saved-image-{j}', 'figure', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f'2c3-label-{j}', 'children', allow_duplicate=True) for j in range(sidebar_len)],
    [Output(f"2c3-feedback","children",allow_duplicate=True)],
    Input('cluster_3_image', 'n_clicks'),
    [Input(f'c3-slot{a}', 'n_clicks') for a in range(8)],
    [Input(f"2c3-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    prevent_initial_call=True
)
def update_sidebar_c3(*args):
    print("\n*************************callback: update_sidebar_c3")
    global saved_imgs, sidebar_outputs
   
    triggered_id = ctx.triggered_id
    print(f'trigger: {triggered_id}')
    if 'cluster' in triggered_id:
        feedback_msg = 'Click on a design to save it'
    elif 'slot' in triggered_id:
        # Handle save_clicked_img_c3 trigger
        cluster = int(ctx.triggered_id[1])
        slot_no = int(ctx.triggered_id.split('slot')[-1])
        set_no = current_set[cluster]
        saved_img_no = first_indices[cluster] + set_no * 8 + slot_no
        
        saved_imgs, feedback_msg = update_saved_imgs(saved_img_no, saved_imgs, sidebar_len)
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
        saved_imgs_source.insert(0, "c3")
        
        feedback_msg = f'Design has been saved '
    elif 'delete' in triggered_id:
        # Handle delete_saved_img trigger
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg = f'Design has been deleted'
    return sidebar_outputs +  [feedback_msg]

# TAB 2-{0}: RIGHT/LEFT CLICKS
@app.callback(
    [Output(f'c0-image{i}-title','children',allow_duplicate=True) for i in range(8)],
    [Input(f'c0-left','n_clicks'),
    Input(f'c0-right','n_clicks')],
    prevent_initial_call=True
)
def c0_update_eight_images(left, right):
    print("\n*************************callback: c0_update_eight_images")
    
    global current_set, sets_explored, function_count   
    direction = ctx.triggered_id.split('-')[1]
          
    if direction=='right':
        current_set[0] = (current_set[0]+1)%(set_count[0])
        function_count[1] += 1
    elif direction=='left':
        current_set[0] = (current_set[0]+set_count[0]-1)%(set_count[0])
        function_count[0] += 1

    # keeping track of all sets explored, for data collection
    set_str = f"c0_set{current_set[0]}"
    if set_str not in sets_explored:
        sets_explored.append(set_str)
#         print(sets_explored)

    current_index = current_set[0]*8
    outputs = [f'Cluster 0 Image {i}: ${all_data["cost"][i]}' for i in range(current_index,current_index+8)]

    return outputs

# TAB 2-{0}: CYCLE THROUGH DESIGN RESERVOIR (NEW!!!!)
@app.callback(
    [Output(f'c0-image{i}','figure') for i in range(8)],
    Input('c0-image0-title','children'),
    prevent_initial_call=True
)
def c0_explore(first_title):
    print("\n*************************callback: c0_explore")
    global current_set
#     print("update images function triggered")
    current_index = current_set[0]*8

    blank_output = {
                    'data': [{'type': 'image', 'z': np.zeros((256, 256, 3), dtype=np.uint8)}],
                    'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}}}

    if current_index + 8 <= (last_indices[0]-first_indices[0]):
        outputs = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(current_index,current_index+8)]
    else: # need to make sure it cuts of instead of spilling over to next cluster
        outputs_one = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(current_index,last_indices[0]+1)]
        leftovers = (last_indices[0]-first_indices[0]+1)%8
        outputs_two = [blank_output for i in range(-leftovers%8)]
        outputs = outputs_one + outputs_two

    return outputs

# TAB 2-{1}: RIGHT/LEFT CLICKS
@app.callback(
    [Output(f'c1-image{i}-title','children',allow_duplicate=True) for i in range(8)],
    [Input(f'c1-left','n_clicks'),
    Input(f'c1-right','n_clicks')],
    prevent_initial_call=True
)
def c1_update_eight_images(left, right):
    print("\n*************************callback: c1_update_eight_images")
    global current_set
    direction = ctx.triggered_id.split('-')[1]

    if direction=='right':
        current_set[1] = (current_set[1]+1)%(set_count[1])
        function_count[1] += 1
    elif direction=='left':
        current_set[1] = (current_set[1]+set_count[1]-1)%(set_count[1])
        function_count[0] += 1

    # keeping track of all sets explored, for data collection
    set_str = f"c1_set{current_set[0]}"
    if set_str not in sets_explored:
        sets_explored.append(set_str)

    current_index = current_set[1]*8
    outputs = [f'Cluster 1 Image {i}: ${all_data["cost"][first_indices[1]+i]}' for i in range(current_index,current_index+8)]
    return outputs

# TAB 2-{1}: CYCLE THROUGH DESIGN RESERVOIR (NEW!!!!)
@app.callback(
    [Output(f'c1-image{i}','figure') for i in range(8)],
    Input('c1-image0-title','children'),
    prevent_initial_call=True
)
def c1_explore(first_title):
    print("\n*************************callback: c1_explore")
    global current_set
    print("update images function triggered")
    current_index = current_set[1]*8
    blank_output = {
                    'data': [{'type': 'image', 'z': np.zeros((256, 256, 3), dtype=np.uint8)}],
                    'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}}}

    if current_index + 8 <= (last_indices[1]-first_indices[1]):
        outputs = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(first_indices[1]+current_index,first_indices[1]+current_index+8)]
    else: # need to make sure it cuts of instead of spilling over to next cluster
        outputs_one = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(first_indices[1]+current_index,last_indices[1]+1)]
#         print(len(outputs_one))
        leftovers = (last_indices[1]-first_indices[1]+1)%8
        outputs_two = [blank_output for i in range(-leftovers%8)]
#         print(len(outputs_two))
        outputs = outputs_one + outputs_two
    return outputs

# TAB 2-{2}: RIGHT/LEFT CLICKS
@app.callback(
    [Output(f'c2-image{i}-title','children',allow_duplicate=True) for i in range(8)],
    [Input(f'c2-left','n_clicks'),
    Input(f'c2-right','n_clicks')],
    prevent_initial_call=True
)
def c2_update_eight_images(left, right):
    print("\n*************************callback: c2_update_eight_images")
    global current_set
    direction = ctx.triggered_id.split('-')[1]

    if direction=='right':
        current_set[2] = (current_set[2]+1)%(set_count[2])
        function_count[1] += 1
    elif direction=='left':
        current_set[2] = (current_set[2]+set_count[2]-1)%(set_count[2])
        function_count[0] += 1

    # keeping track of all sets explored, for data collection
    set_str = f"c2_set{current_set[0]}"
    if set_str not in sets_explored:
        sets_explored.append(set_str)

    current_index = current_set[2]*8
    outputs = [f'Cluster 2 Image {i}: ${all_data["cost"][first_indices[2]+i]}' for i in range(current_index,current_index+8)]
    return outputs

# TAB 2-{2}: CYCLE THROUGH DESIGN RESERVOIR (NEW!!!!)
@app.callback(
    [Output(f'c2-image{i}','figure') for i in range(8)],
    Input('c2-image0-title','children'),
    prevent_initial_call=True
)
def c2_explore(first_title):
    print("\n*************************callback: c2_explore")
    global current_set
    print("update images function triggered")
    current_index = current_set[2]*8
    print(current_index)
    blank_output = {
                    'data': [{'type': 'image', 'z': np.zeros((256, 256, 3), dtype=np.uint8)}],
                    'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}}}

    if current_index + 8 <= (last_indices[2]-first_indices[2]):
        outputs = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(first_indices[2]+current_index,first_indices[2]+current_index+8)]
    else: # need to make sure it cuts of instead of spilling over to next cluster
        outputs_one = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(first_indices[2]+current_index,last_indices[2]+1)]
        leftovers = (last_indices[2]-first_indices[2]+1)%8
        outputs_two = [blank_output for i in range(-leftovers%8)]
        outputs = outputs_one + outputs_two
    return outputs

# TAB 2-{3}: RIGHT/LEFT CLICKS
@app.callback(
    [Output(f'c3-image{i}-title','children',allow_duplicate=True) for i in range(8)],
    [Input(f'c3-left','n_clicks'),
    Input(f'c3-right','n_clicks')],
    prevent_initial_call=True
)
def c3_update_eight_images(left, right):
    print("\n*************************callback: c3_update_eight_images")
    global current_set
    direction = ctx.triggered_id.split('-')[1]

    if direction=='right':
        current_set[3] = (current_set[3]+1)%(set_count[3])
        function_count[1] += 1
    elif direction=='left':
        if current_set[3]==0:
            return [no_update for i in range(8)]
        current_set[3] = (current_set[3]+set_count[3]-1)%(set_count[3])
        function_count[0] += 1

    # keeping track of all sets explored, for data collection
    set_str = f"c3_set{current_set[0]}"
    if set_str not in sets_explored:
        sets_explored.append(set_str)

    current_index = current_set[3]*8
    outputs = [f'Cluster 3 Image {i}: ${all_data["cost"][first_indices[3]+i]}' for i in range(current_index,current_index+8)]
    return outputs

# TAB 2-{3}: CYCLE THROUGH DESIGN RESERVOIR (NEW!!!!)
@app.callback(
    [Output(f'c3-image{i}','figure') for i in range(8)],
    Input('c3-image0-title','children'),
    prevent_initial_call=True
)
def c3_explore(first_title):
    print("\n*************************callback: c3_explore")
    global current_set
    print("update images function triggered")
    current_index = current_set[3]*8
    print(current_index)
    blank_output = {
                    'data': [{'type': 'image', 'z': np.zeros((256, 256, 3), dtype=np.uint8)}],
                    'layout':{'margin': '0px', 'padding': '0px', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}}}

    if current_index + 8 <= (last_indices[3]-first_indices[3]):
        outputs = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(first_indices[3]+current_index,first_indices[3]+current_index+8)]
    else: # need to make sure it cuts of instead of spilling over to next cluster
        outputs_one = [make_figure(255-(all_data['px_data'][i,:,:,0]*255),300) for i in range(first_indices[3]+current_index,last_indices[3]+1)]
        leftovers = (last_indices[3]-first_indices[3]+1)%8
        outputs_two = [blank_output for i in range(-leftovers%8)]
        outputs = outputs_one + outputs_two
    return outputs


# ********************** TAB INTER *******************************
@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'{i}-to-inter','n_clicks') for i in ["1","2c0","2c1","2c2","2c3","3","4","lv","5","6","fin"]], # sv edit
    prevent_initial_call=True
)
def display_tab_inter(*args):
    print('\n*************************callback: display_tab_inter')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "inter"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-inter'

# ********************** TAB 3 (RENDER) *******************************
# TAB 3: OPEN
@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'{i}-to-3','n_clicks') for i in ["1","2c0","2c1","2c2","2c3","inter","4","lv","5","6","fin"]], 
    prevent_initial_call=True
)
def display_tab_3(*args):
    print('\n*************************callback: display_tab_3')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "3d"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-3'

# TAB 3: UPDATE DROPDOWN OPTIONS AFTER OPENING TAB OR DELETING IMAGE
@app.callback(
    Output("3d-dropdown","options", allow_duplicate=True), 
    Output("3-feedback", "children", allow_duplicate=True),
    [Input(f'{i}-to-3','n_clicks') for i in ["1","2c0","2c1","2c2","2c3","inter","4","lv","5","6","fin"]],
    [Input(f"3-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    prevent_initial_call=True
)
def update_dropdown_options_3(*args): # this replaces " add_new_image " 
    print("\n*************************callback: update_dropdown_options_3")
    global saved_imgs, sidebar_outputs
    triggered_id = ctx.triggered_id
    if 'delete' in triggered_id:
        # Handle delete_saved_img trigger
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])

    dropdown_options = update_dropdown_options(saved_imgs)
    feedback_msg = 'Select a saved design to view in 3D'
  
    return dropdown_options, feedback_msg


#TAB 3: UPDATE SIDEBAR AFTER OPENING TAB OR DELETING IMAGE 

@app.callback(
    [Output(f'3-saved-image-{j}', 'figure', allow_duplicate = True) for j in range(sidebar_len)],
    [Output(f'3-label-{j}', 'children', allow_duplicate = True) for j in range(sidebar_len)],  
    Input("3d-dropdown","options"),
    prevent_initial_call=True
)
def update_sidebar_3(*args):
    print('\ncallback: update_sidebar_3')
  
    return sidebar_outputs

#TAB 3: CLICK IMAGE FROM SIDEBAR & UPDATE DROPDOWN VALUE
@app.callback(
    Output('3d-dropdown','value'),
    Output('3-feedback','children', allow_duplicate=True),
    [Input(f'3-container-{j}','n_clicks') for j in range(sidebar_len)],
    prevent_initial_call=True
)
def click_sidebar_image_3(*args): # was ' choose_3d_image '
    print('\n*************************callback: click_sidebar_image_3')
    try:
        clicked_saved_img = int(ctx.triggered_id.split('-')[-1])
        feedback_msg = f'Saved Design in 3D'
    except (ValueError, IndexError):
        print("Empty Saved Image Slot")
        return no_update, no_update
    # not sure why it still returns indexerror
    else:
        return saved_imgs[clicked_saved_img], feedback_msg #saved_imgs[len(saved_imgs)-clicked_saved_img-1], feedback_msg


# TAB 3: RENDER IMAGE

@app.callback(
    Output('3d-render','figure'),
    Input('3d-dropdown','value'),
    Input('extrude-button','n_clicks'),
    Input('revolve-button','n_clicks'),
    prevent_initial_call=True
)
def render_3d_view(value,ext,rev):
    print("\n*************************callback: render_3d_view")
    global function_count
    if value == None:
        return no_update
    elif ctx.triggered_id == 'revolve-button':
        plot_figure = revolve_3d(all_data['px_data'][value])
    else:
        plot_figure = render_3d(all_data['px_data'][value])

    function_count[0] += 1

    return plot_figure


# **************************** TAB 4 & 4-2*******************************
# TAB 4 & 4-2 : OPEN TAB

@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'{i}-to-4','n_clicks') for i in ["inter","lv","6"]],
    prevent_initial_call=True
)
def display_tab_4(*args):
    print('\n*************************callback: display_tab_4')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "adjust"
    tab_start = time.time() # start measuring time spent (for next tab)
    
    return 'tab-4'

@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'{i}-to-lv','n_clicks') for i in ["inter","4"]],
    prevent_initial_call=True
)
def display_tab_4_2(*args):
    print('\n*************************callback: display_tab_4_2')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "lv"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-4-2'

# TAB 4 & 4-2: UPDATE DROPDOWN OPTIONS AFTER OPENING TAB, SAVING IMAGE, or DELETING IMAGE
    
@app.callback(
    Output('adjust-dropdown','options', allow_duplicate=True), 
    Output('4-feedback','children',allow_duplicate=True),
    [Input(f'{i}-to-4','n_clicks') for i in ["inter","lv","6"]],
    [Input(f"4-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
     Input('adjust-save-button','n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_options_4(*args): # this replaces " add_new_image " 
    print("\n*************************callback: update_dropdown_options_4")
#     print("save button clicked")
    # add image to saved_imgs list and main_dict
    global saved_imgs, saved_imgs_source, all_data, current_lv_data, current_cond_data, current_cost, current_image, sidebar_outputs, graphs
   
    triggered_id = ctx.triggered_id
   
    if triggered_id in ['inter-to-4', 'lv-to-4', '6-to-4']: # when tab is opened
        feedback_msg = 'Click a design to edit it'
        

    elif 'delete' in triggered_id:
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg = f'Design has been deleted'
        else:
            feedback_msg = 'There is no design to delete'
    else:
        next_index = len(all_data['dims'])
        saved_imgs, feedback_msg = update_saved_imgs(next_index,saved_imgs,sidebar_len)
        
        # record which tab the saved img was saved
        saved_imgs_source.insert(0,"4")
        
        # update all_data with new entry
        all_data  = update_all_data(all_data, current_image, current_lv_data, current_cond_data, current_cost, current_dimensions)
       
        # need to reshape back to (128,96) for other functions
        current_image = np.reshape(current_image, (128,96))
        print(f'current_image shape: {np.shape(current_image)}')
        
        # generate sidebar outputs
        image_data=255-(all_data['px_data'][-1][:,:,0]*255) # last px_data entry
        graph = dcc.Graph(figure=make_figure(image_data))
        graphs.append(graph) # why was this not in 'global' before if it's being updated??
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])  
        
        """
        print(f'current_lv_data: {current_lv_data}')
        print(f'current_cond_data: {current_cond_data}')
        print(f'current_image: {np.shape(current_image)}')
        plt.imshow(current_image, cmap='gray_r')
        plt.show()
        """

    dropdown_options = update_dropdown_options(saved_imgs)
    
    return dropdown_options , feedback_msg

    
@app.callback(
    [Output('lv-dropdown','options', allow_duplicate=True)], 
    [Output('lv-feedback', 'children', allow_duplicate=True)],
    [Output(f'lv_{i}','value') for i in list(range(1,latent_dim+1))], # to trigger first "if" statemtn of update_sliders
    [Input(f'{i}-to-lv','n_clicks') for i in ["inter","4"]],
    [Input(f"lv-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    Input('lv-save-button','n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_options_4_2(*args): # this replaces " add_new_image " 
    print("\n*************************callback: update_dropdown_options_4_2")
#     print("save button clicked")
    # add image to saved_imgs list and main_dict
    global saved_imgs, saved_imgs_source, all_data, current_lv_data, current_cond_data, current_cost, current_image, sidebar_outputs, graphs
    
    if len(current_lv_data) == 0:
        dropdown_options = update_dropdown_options(saved_imgs)
        feedback_msg = 'Click on a saved design to edit it'
        return [dropdown_options]+ [feedback_msg]+ [no_update for i in range(latent_dim)]
    
    triggered_id = ctx.triggered_id

    if triggered_id in ['4-to-lv','inter-to-lv']:
        feedback_msg = 'Click on a saved design to edit it'
    
    elif 'delete' in triggered_id:
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg = f'Design has been deleted'
        else:
            feedback_msg = 'There is no design to delete'
    else:
        next_index = len(all_data['dims'])
        saved_imgs, feedback_msg = update_saved_imgs(next_index,saved_imgs,sidebar_len)
        
        # record which tab the saved img was saved
        saved_imgs_source.insert(0,"lv")
       
        # update all_data with new entry
        all_data  = update_all_data(all_data, current_image, current_lv_data, current_cond_data, current_cost, current_dimensions)
        
        # change back to (128,96) for other functions
        current_image = np.reshape(current_image, (128,96))
        
        # generate sidebar outputs
        image_data=255-(all_data['px_data'][-1][:,:,0]*255) # last px_data entry
        graph = dcc.Graph(figure=make_figure(image_data))
        graphs.append(graph) # why was this not in 'global' before if it's being updated??
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])  
    
    
    dropdown_options = update_dropdown_options(saved_imgs)
    return [dropdown_options]+ [feedback_msg]+ list(current_lv_data)


#TAB 4 & 4-2: UPDATE SIDEBAR AFTER DROPDOWN OPTIONS UPDATED

@app.callback(
    [Output(f'4-saved-image-{j}', 'figure', allow_duplicate = True) for j in range(sidebar_len)],
    [Output(f'4-label-{j}', 'children', allow_duplicate = True) for j in range(sidebar_len)],
    Input("adjust-dropdown","options"),
    prevent_initial_call=True
)
def update_sidebar_4(*args):
    print('\n*************************callback: update_sidebar_4')
    
    return sidebar_outputs

@app.callback(
    [Output(f'lv-saved-image-{j}', 'figure', allow_duplicate = True) for j in range(sidebar_len)],
    [Output(f'lv-label-{j}', 'children', allow_duplicate = True) for j in range(sidebar_len)],
    Input('lv-dropdown','options'),
    prevent_initial_call=True
)
def update_sidebar_4_2(*args):
    print('\n*************************callback: update_sidebar_4_2')
    return sidebar_outputs
  

# TAB 4 & 4-2: CLICK IMAGE FROM SIDEBAR 
@app.callback(
    Output('adjust-dropdown','value', allow_duplicate = True),
    [Input(f'4-container-{j}','n_clicks') for j in range(sidebar_len)],
    prevent_initial_call=True
)
def click_sidebar_image_4(*args): # was ' choose_adjust_image '
    print('\n*************************callback: click_sidebar_image_4')
    global saved_imgs
    try:
        clicked_saved_img = int(ctx.triggered_id.split('-')[-1])
    except (ValueError, IndexError):
        print("Empty Saved Image Slot")
        return no_update
    # not sure why it still returns indexerror
    else:
        return saved_imgs[clicked_saved_img] #saved_imgs[len(saved_imgs)-clicked_saved_img-1]
    
@app.callback(
    Output('lv-dropdown','value', allow_duplicate = True),
    [Input(f'lv-container-{j}','n_clicks') for j in range(sidebar_len)],
    prevent_initial_call=True
)
def click_sidebar_image_4_2(*args): # was ' choose_adjust_image '
    print('\n*************************callback: click_sidebar_image_4_2')
    global saved_imgs
    try:
        clicked_saved_img = int(ctx.triggered_id.split('-')[-1])
    except (ValueError, IndexError):
        print("Empty Saved Image Slot")
        return no_update
    # not sure why it still returns indexerror
    else:
        return saved_imgs[clicked_saved_img] #saved_imgs[len(saved_imgs)-clicked_saved_img-1]
    
@app.callback(
    [Output(f'lv_{i}', 'value',allow_duplicate=True) for i in (list(range(1,latent_dim+1)) + ['vf','sw','sh'] )],
    [Output('adjust-dim','children',allow_duplicate=True)],
    Input('adjust-dropdown', 'value'),
    prevent_initial_call=True
)

# TAB 4 & 4-2: UPDATE DROPDOWN VALUE

def update_dropdown_value_4(value):
    print("\n*************************callback: update_dropdown_value_4")
    # Choosing an image from the sidebar will update cond. input sliders & lv sliders
    global current_lv_data, current_cond_data, current_image
    if value == None:
        return [no_update for i in range(15)]
    elif value >= 0:
        num_images=1
        current_image = all_data["px_data"][value,:,:,0] # update current_image when loaded from sidebar


        cond_inp = all_data['cond_inputs'][value]
        current_cond_data = copy.deepcopy(cond_inp)
        cond_inp.resize((1,latent_dim),refcheck=False)

        noise=all_data['lv_data'][value]
        current_lv_data = copy.deepcopy(noise)
        noise.resize((1,latent_dim),refcheck=False)

        cond_output = [round(cond_inp[0][0],2),round(cond_inp[0][4],2),round(cond_inp[0][8],2)]
        lv_output = [round(noise[0][i],2) for i in range(latent_dim)]

        # retrieve dimensions
        dimensions = all_data['dims'][value]
        current_dimensions = dimensions
        dimensions_in = [round(i/25.4,2) for i in dimensions]
        dim_output_str = [f"Height: {dimensions_in[0]}in\nSeat Width: {dimensions_in[1]}in\nLeg Width: {dimensions_in[2]}in\nHeight:Width Ratio: {round(dimensions[3],2)}"]

        final_output =  lv_output + cond_output + dim_output_str
        
        # reshape current_image back to 128,96
        current_image = np.reshape(current_image, (128,96))
        
        
        """
        print(f'current_lv_data: {current_lv_data}')
        print(f'current_cond_data: {current_cond_data}')
        print(f'current_image: {np.shape(current_image)}')
        plt.imshow(current_image, cmap='gray_r')
        plt.show()
        """

        
        return final_output

    else:
        return [no_update for i in range(15)]
    
#TAB 4 & 4-2: SYNCING CHOSEN IMAGE

@app.callback(
    Output('adjust-dropdown','value',allow_duplicate=True),
    Output('suggest-dropdown','value',allow_duplicate=True),
    Input('lv-dropdown','value'),
    prevent_initial_call=True
)
def sync_adjust_lv(value):
    print("\n*************************callback: sync_adjust_lv")
    # choosing image in lv tab changes adjust tab dropdown as well, triggering update_image_from_dropdown function
    return value,value


# TAB 4  & 4-2: ADJUST IMAGE WITH SLIDERS
@app.callback(
    Output('adjust-image','figure',allow_duplicate=True), # Any change in lv slider values will trigger image refresh
    Output('lv-image','figure',allow_duplicate=True),
    Output('interp-slider','value',allow_duplicate=True),
    Output('adjust-dim','children',allow_duplicate=True),
    Output('lv-dim','children',allow_duplicate=True),
    Output('lv-lca','children',allow_duplicate=True),
    Output('adjust-lca','children',allow_duplicate=True),
    Output('4-feedback','children',allow_duplicate=True),
    [Input(f'lv_{i}', 'value') for i in (['vf','sw','sh'] + list(range(1,latent_dim+1)))],
    prevent_initial_call=True
)
def update_sliders(vf,sw,sh,lv_1,lv_2,lv_3,lv_4,lv_5,lv_6,lv_7,lv_8,lv_9,lv_10,lv_11,lv_12):
    print("\n*************************callback: update_sliders")
# def update_sliders(vf, sw, sh):
    global current_lv_data, current_cond_data, current_image, current_cost, current_dimensions, function_count, interp_imgs, interp_bcs, interp_noise
    
    # if triggerd id is vf,sw,sh, then use current_lv_data for cond list
    # if triggerd id is any lv, then do normal
    # all other functions that update cond inps/lvs, make them output lvs first, then cond inps
 
    num_images=1

   
    triggered_id = ctx.triggered_id
    print(f'triggered_id: {triggered_id}')
          
    # keeping track of # conditional/LV function was triggered   
    input_id = ctx.triggered_id[-2:]
    function_count[0] += 1

    # save current cond. input values
    cond_list = [vf, vf, vf, vf, sw, sw, sw, sw, sh, sh, sh, sh]
    cond_inp = np.array(cond_list)
    current_cond_data = copy.deepcopy(cond_inp)
    cond_inp.resize((1,latent_dim),refcheck=False)
    
    # separated lv generation into if these statements b/c of pixclick
    # basically bc pixclick doesnt update the lv sliders, only the global variable
    # so if we didnt do this, then if u move vf/sw/sh slider after using 'selected area slider'
    # it would use the old lvs (instead of new ones for the new pixclick img)
    # and cond inps dont matter b/c pixclick keeps same cond inps
    # there should be a less hackey way of doing this tho...
    print(f'current_lv_data: {np.shape(current_lv_data)}')
    
    if triggered_id in ['lv_vf', 'lv_sw', 'lv_sh']: 
        # basically this is triggered if you manually moved cond inp sliders
        current_lv_data = np.array(current_lv_data)
        noise = current_lv_data.copy()
        noise.resize((1,latent_dim),refcheck=False)
    else:
        # this is triggered if you moved lv sliders OR if another callback triggered
        # update of lvs + vf (like dropdown/sidebar)
        # this is b/c i set up the other callbacks so that the first outputs are lvs (not cond inps)
        lv_list=[lv_1,lv_2,lv_3,lv_4,lv_5,lv_6,lv_7,lv_8,lv_9, lv_10,lv_11,lv_12]
        noise=np.array(lv_list)
        current_lv_data = np.array(lv_list)
        noise.resize((1,latent_dim),refcheck=False)
        
    print(f'current_lv_data: {np.shape(current_lv_data)}')
    # generate image based on changed cond.input & lv values
    _, int_gen_imgs =generate_imgs(num_images,noise,cond_inp, gen_model)
    current_image = int_gen_imgs[0,:,:,0]
    img_data=255-(int_gen_imgs[:,:,:,0]*255)
    
    # change pixclick so all interpolations = current image
    print(f'current_lv_data: {np.shape(current_lv_data)}')
    interp_imgs = np.repeat(int_gen_imgs, num_interp, axis=0)
    noise = current_lv_data.copy()
    noise.resize((1,latent_dim),refcheck=False)
    interp_noise = np.repeat(noise, num_interp, axis=0)
    bcs = current_cond_data.copy()
    bcs.resize((1,latent_dim),refcheck = False)
    interp_bcs = np.repeat(bcs, num_interp, axis=0)
    interp_imgs = np.insert(interp_imgs, 0, interp_imgs[0], axis=0)
    interp_noise = np.insert(interp_noise, 0, interp_noise[0], axis=0)
    interp_bcs = np.insert(interp_bcs, 0, interp_bcs[0], axis=0)
    print(f'shapes interp imgs, bcs, noise: {np.shape(interp_imgs)}, {np.shape(interp_bcs)}, {np.shape(interp_noise)}')
    
    
    print(f'input to make_figure is img_data[0]: {np.shape(img_data[0])}')
    print(f'img_data {np.shape(img_data)}')
    updated_fig = make_figure(img_data[0])

    # compute updated dimensions
    dimensions = compute_dimensions(img_data)
    current_dimensions = dimensions
    dimensions_in = [round(i/25.4,2) for i in dimensions]
    # CONVERT UNITS TO CM/IN
    dim_output_str = f"Height: {dimensions_in[0]}in\nSeat Width: {dimensions_in[1]}in\nLeg Width: {dimensions_in[2]}in\nHeight:Width Ratio: {round(dimensions[3],2)}"

    # retrieve lca results
    lca_metrics = frameworkCalcs(int_gen_imgs[0,:,:,0])
#     equivMiles = round(lca_metrics[-1])
    equivCost = lca_metrics[-1]
    current_cost = equivCost
#     lca_output_str = f"By creating this design, you can save CO2 equivalent of driving {equivMiles} miles."
    lca_output_str = f"Printing this design costs ${equivCost}"

    feedback_msg = "Move the sliders to adjust your design"
#     pixclick_msg = "You can drag a rectangle on the image to edit a specific area"
        
    """
    print(f'current_lv_data: {current_lv_data}')
    print(f'current_cond_data: {current_cond_data}')
    print(f'current_image: {np.shape(current_image)}')
    plt.imshow(current_image, cmap='gray_r')
    plt.show()
    """

    return updated_fig, updated_fig,0,dim_output_str, dim_output_str, lca_output_str, lca_output_str, feedback_msg


# TAB 4: PIXCLICK BEHIND-THE-SCENE GENERATION
@app.callback(
    Output("pixclick-feedback", "children"),
    Output("interp-slider", "value"),
    Input("adjust-image", "relayoutData"),
    prevent_initial_call=True
)
def pixclick_trigger(relayout_data):
    print("\n*************************callback: pixclick_trigger")
    global current_lv_data, current_cond_data, current_image, interp_imgs, interp_noise, interp_bcs
#     print(f'relayout_data: {relayout_data}')
    try:
        # make sure it only runs when rectangle is dragged
        shape_dict = relayout_data["shapes"][0] 
    
    except:
        # KeyError will arise when image is chosen, bypass that error
        print('pixclick exception')
        return "You can drag a rectangle on the image to edit a specific area", no_update
    else:
        print('pixclick triggered')
        current_image = np.reshape(current_image, (128,96))
        # Get rectangle coordinates
        x0, x1, y0, y1 = shape_dict["x0"], shape_dict["x1"], shape_dict["y0"], shape_dict["y1"]

        # include error correction if area is too small/rectangle not dragged to x or y
        selected_area = (round(y0), round(y1), round(x0), round(x1))

        # Create several noise_variations
#         print(f'current_lv_data: {np.shape(current_lv_data)}')
        noise_variations = adjust_lvs(current_lv_data, latent_range, num_variations)

        # Generate image variations
        feature_imgs = adjust_imgs(noise_variations, current_cond_data, num_variations, gen_model)
        
#         print('EXPECTED CURRENT IMAGE:')
#         plt.imshow(current_image,cmap='gray_r')
#         plt.show()
        
        
#         print('VARIATIONS:')
#         features_fig, axes = plt.subplots(1, len(feature_imgs), figsize=(20, 10))
#         for i in range(len(feature_imgs)):
#             axes[i].imshow(feature_imgs[i], cmap='gray_r')  # You can adjust the color map as per your image type
#             #axes[i].axis('off')  # Turn off axis labels and ticks
#         plt.show()
        

        # Find number of differences in selected area and in total area minus selected
        #print(f'SELECTED AREA: {selected_area}')
        all_ratios, area_ratios = find_black_counts(current_image, feature_imgs, selected_area, num_variations)
     
        # Find similarity score between images & original (based on discriminator feature vectors)
        dist_scores = feature_dist(current_image, current_cond_data, feature_imgs, disc_model, num_variations)
      
        # Calculate final scores based off all_ratios, area_ratios, dist_scores & return best latent idx
        lv_idx, final_scores = get_lv_idx(all_ratios, area_ratios, dist_scores)
        
        
#         print('ALL & AREA & DIST & FINAL:')
#         print([np.round(ratio, 2) for ratio in all_ratios ])
#         print([np.round(ratio, 2) for ratio in area_ratios ])
#         print([np.round(dist, 2) for dist in dist_scores ])
#         print([np.round(score, 2) for score in final_scores ])
#         print(f'LV_IDX: {lv_idx}')
        
        
        if lv_idx:
    
            interp_imgs, interp_noise = gen_interp_imgs(feature_imgs,current_lv_data, current_cond_data, noise_variations, lv_idx, num_interp, latent_stretch, gen_model)
            print(f'shapes interp imgs, bcs, noise: {np.shape(interp_imgs)}, {np.shape(interp_bcs)}, {np.shape(interp_noise)}')
            
            ############
#             print('INTERPOLATIONS:')
#             features_fig, axes = plt.subplots(1, len(interp_imgs), figsize=(20, 2))
#             for i in range(len(interp_imgs)):
#                 axes[i].imshow(interp_imgs[i], cmap='gray_r')  # You can adjust the color map as per your image type
#                 #axes[i].axis('off')  # Turn off axis labels and ticks
#             plt.show()
            ##########
            return "Move the Selected Area slider to see the changes!", 0 #, no_update


            """
            print(f'current_lv_data: {current_lv_data}')
            print(f'current_cond_data: {current_cond_data}')
            print(f'current_image: {np.shape(current_image)}')
            plt.imshow(current_image, cmap='gray_r')
            plt.show()
            """

        else:
            
            return "Function incompatible with selected area. Please select a different area!", no_update

# TAB 4: PIXCLICK SELECTED AREA SLIDER
@app.callback(
    [Output("adjust-image","figure")],
    [Output("lv-image","figure")],
    #[Output(f'lv_{i}','value') for i in list(range(1,latent_dim+1))],
    Input("interp-slider", "value"),
    State("adjust-image", "relayoutData"),
    prevent_initial_call=True
)
def change_current_img(slider,relayout_data):
    global current_image, current_lv_data, interp_imgs, interp_noise
    print("\n*************************callback: change_current_img")
#     print(f'relayout data: {relayout_data}')
    try:
        shape_dict = relayout_data["shapes"][0]
        #print(f'shape_dict: {shape_dict}')
    
    except:     
#         print('returning no_updates b/c shape error')
        return [no_update, no_update]
    else: 
        if slider !=0:
            #print('displaying new img from pixclick')
            
            function_count[1] += 1
            current_image = interp_imgs[slider, :,:,0]
            print(f'current_image: {np.shape(current_image)}')
            current_img = np.expand_dims(current_image, axis=0)
            print(f'current_img: {np.shape(current_img)}')
            current_noise = interp_noise[slider]
            print(f'current_noise: {np.shape(current_noise)}')
            current_noise = np.expand_dims(current_noise, axis = 0)
            print(f'current_noise: {np.shape(current_noise)}')
            current_lv_data = [current_noise[0,i] for i in range(latent_dim)]
           
            current_img=255-(current_img*255) # reverse BW: don't want this to carry over to other functions
            print(f'input to make figure is current_img[0]: {np.shape(current_img[0])}')
            print(f'current_img: {np.shape(current_img)}')
            fig = make_figure(current_img[0])
            figures = [fig,fig]

            """
            print(f'current_lv_data: {current_lv_data}')
            print(f'current_cond_data: {current_cond_data}')
            print(f'current_image: {np.shape(current_image)}')
            plt.imshow(current_image, cmap='gray_r')
            plt.show()
            """
            return figures
        else:
#             print('returning no_updates bc slider =0')
            return [no_update, no_update]




# ************************* TAB 5: BLEND FUNCTION ************************
# TAB 5: OPEN 
@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    Input('inter-to-5','n_clicks'),
    prevent_initial_call=True
)
def display_tab_5(*args):
    print('\n*************************callback: display_tab_5')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "blend"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-5'

# TAB 5: UPDATE DROPDOWN OPTIONS AFTER OPENING TAB, SAVING IMAGE, OR DELETING IMAGE
@app.callback(
    [Output("blend-dropdown-1","options", allow_duplicate=True),
    Output("blend-dropdown-2","options", allow_duplicate=True)], 
    [Output("5-feedback","children",allow_duplicate=True)],
    Input('inter-to-5','n_clicks'),
    [Input(f"5-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    Input('blend-save-button','n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_options_5(*args): # this replaces " add_new_image " 
    print("\n*************************callback: update_dropdown_options_5")
#     print("save button clicked")
    # add image to saved_imgs list and main_dict
    global saved_imgs, all_data, current_lv_data, current_lv_data, current_cond_data, current_cost, current_image, sidebar_outputs, graphs
   
    triggered_id = ctx.triggered_id
  
    if triggered_id == 'inter-to-5':
        feedback_msg= "Click a design to save it"
    
    elif 'delete' in triggered_id:
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg = f'Design has been deleted'
        else:
            feedback_msg = 'There is no design to delete'
    else:
        next_index = len(all_data['dims'])

        saved_imgs, feedback_msg = update_saved_imgs(next_index,saved_imgs,sidebar_len)
   
        # record which tab the saved img was saved
        saved_imgs_source.insert(0,"5")
    
        # update all_data with new entry
        all_data  = update_all_data(all_data, current_image, current_lv_data, current_cond_data, current_cost, current_dimensions)
        
        # change back to (128,96)
        current_image = np.reshape(current_image, (128,96))
        
        # generate sidebar outputs
        image_data=255-(all_data['px_data'][-1][:,:,0]*255) # last px_data entry
        graph = dcc.Graph(figure=make_figure(image_data))
        graphs.append(graph) # why was this not in 'global' before if it's being updated??
        sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])  
        
        """
        print(f'current_lv_data: {current_lv_data}')
        print(f'current_cond_data: {current_cond_data}')
        print(f'current_image: {np.shape(current_image)}')
        plt.imshow(current_image, cmap='gray_r')
        plt.show()
        """

    dropdown_options = update_dropdown_options(saved_imgs)
    dropdown_options = [dropdown_options for i in range(2)]   
    return dropdown_options+ [feedback_msg]


#TAB 5: UPDATE SIDEBAR AFTER DROPDOWN OPTIONS ARE UPDATED

@app.callback(
    [Output(f'5-saved-image-{j}', 'figure', allow_duplicate = True) for j in range(sidebar_len)],
    [Output(f'5-label-{j}', 'children', allow_duplicate = True) for j in range(sidebar_len)],
    Input("blend-dropdown-1","options"),
    prevent_initial_call=True
)
def update_sidebar_5(*args):
    print('\n*************************callback: update_sidebar_5')
    
    return sidebar_outputs

#TAB 5: CLICK IMAGE FROM SIDEBAR & UPDATE DROPDOWN VALUE

switch = False
@app.callback(
    [Output(f'blend-dropdown-{i}','value', allow_duplicate = True) for i in [1,2]],
    [Output('5-feedback','children', allow_duplicate = True)],
    [Input(f'5-container-{j}','n_clicks') for j in range(sidebar_len)],
    prevent_initial_call=True
)
def click_sidebar_image_5(*args): # was ' choose_blend_image '
    print('\n*************************callback: click_sidebar_image_5')
    global switch
    try:
        clicked_saved_img = int(ctx.triggered_id.split('-')[-1])
    except (ValueError, IndexError):
        print("Empty Saved Image Slot")
        return no_update
    # not sure why it still returns indexerror
    switch = not switch
    if switch: # Left
        feedback_msg = 'First image selected.'
        return [saved_imgs[clicked_saved_img], no_update, feedback_msg] #[saved_imgs[len(saved_imgs)-clicked_saved_img-1], no_update, feedback_msg]
    else:
        feedback_msg = 'Second image selected. User the slider to blend between chosen images.'
        return [no_update, saved_imgs[clicked_saved_img], feedback_msg] #[no_update, saved_imgs[len(saved_imgs)-clicked_saved_img-1], feedback_msg]

# TAB 5: UPDATE DROPDOWN VALUE

@app.callback(
    Output("blended-image","figure"),
    Output("blend-dim","children"),
    Output("blend-lca","children"),
    Input("blend-dropdown-1","value"),
    Input("blend-dropdown-2","value"),
    Input("blender", "value"),
    prevent_initial_call=True
)
def update_dropdown_value_5(dropdown_1, dropdown_2, slider): # was " blend_function"
    print("\n*************************callback: update_dropdown_value_5")
    global int_noise, int_cond, img_data, int_gen_imgs, int_comp_scores, current_dimensions, function_count


    if dropdown_1 == None or dropdown_2 == None:
        #print("callback19-1")
        return no_update, no_update, no_update

    elif ctx.triggered_id != "blender": # if a dropdown menu is selected:
        lvdata_1 = all_data['lv_data'][dropdown_1]
        lvdata_2 = all_data['lv_data'][dropdown_2]
        # generating interpolated images
        cond_1 = all_data['cond_inputs'][dropdown_1]
        cond_2 = all_data['cond_inputs'][dropdown_2]

        num_images=10
        # generate 10 interpolated points
        print(f'lvdata_1: {np.shape(lvdata_1)}')
        int_noise=interpolate_points(lvdata_1, lvdata_2, n_steps=num_images)
        int_noise.resize((num_images,latent_dim),refcheck=False)

        int_cond = interpolate_points(cond_1, cond_2, n_steps=num_images)
        int_cond.resize((num_images, latent_dim),refcheck = False)

        _, int_gen_imgs =generate_imgs(num_images,int_noise,int_cond, gen_model)
        img_data=255-(int_gen_imgs[:,:,:,0]*255)
        updated_fig = make_figure(img_data[0])

        # compute updated dimensions
        dimensions = all_data['dims'][dropdown_1]
        current_dimensions = dimensions
        dimensions_in = [round(i/25.4,2) for i in dimensions]
        dim_output_str = f"Height: {dimensions_in[0]}in\nSeat Width: {dimensions_in[1]}in\nLeg Width: {dimensions_in[2]}in\nHeight:Width Ratio: {round(dimensions[3],2)}"
        # retrieve lca results
        lca_metrics = frameworkCalcs(int_gen_imgs[:,:,:,0][slider])
#         equivMiles = round(lca_metrics[-1])
        equivCost = lca_metrics[-1]
    #     lca_output_str = f"By creating this design, you can save CO2 equivalent of driving {equivMiles} miles."
        lca_output_str = f"Printing this design costs ${equivCost}"

    else: # if the slider is moved
        function_count[0] += 1

        updated_fig = make_figure(img_data[slider])
        # store current image data as global variables to access from save button callback function
        global current_lv_data, current_cond_data, current_image, current_cost
        current_lv_data = int_noise[slider]
        current_cond_data = int_cond[slider]
        current_image = int_gen_imgs[:,:,:,0][slider]
       
        #current_image = current_image[np.newaxis,:]

        # display updated dimensions
        dimensions = compute_dimensions(img_data[slider])
        current_dimensions = dimensions
        dimensions_in = [round(i/25.4,2) for i in dimensions]
        dim_output_str = f"Height: {dimensions_in[0]}in\nSeat Width: {dimensions_in[1]}in\nLeg Width: {dimensions_in[2]}in\nHeight:Width Ratio: {round(dimensions[3],2)}"

        # retrieve lca results
        lca_metrics = frameworkCalcs(int_gen_imgs[:,:,:,0][slider])
#         equivMiles = round(lca_metrics[-1])
        equivCost = lca_metrics[-1]
        current_cost = equivCost
#     lca_output_str = f"By creating this design, you can save CO2 equivalent of driving {equivMiles} miles."
        lca_output_str = f"Printing this design costs ${equivCost}"
        
        """
        print(f'current_lv_data: {current_lv_data}')
        print(f'current_cond_data: {current_cond_data}')
        print(f'current_image: {np.shape(current_image)}')
        plt.imshow(current_image, cmap='gray_r')
        plt.show()
        """
    #feedback_msg = "Toggle the slider to interpolate between the chosen images."
    return updated_fig, dim_output_str, lca_output_str


# ********************* TAB 6 (SUGGEST) *************************

# TAB 6 OPEN
@app.callback(  
    Output('tabs-example','value',allow_duplicate=True),
    [Input('inter-to-6','n_clicks'),
    Input('4-to-6','n_clicks')],
    prevent_initial_call=True
)
def display_tab_6(*args):
    print('\n*************************callback: display_tab_6')
    global tab_start, tab_end, current_tab, usage_log, function_count
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "suggest"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-6'


# TAB 6: UPDATE DROPDOWN OPTIONS AFTER OPENING TAB, SAVING IMAGE OR DELETING IMAGE
@app.callback(
    Output('suggest-dropdown','options', allow_duplicate=True),
    Input('inter-to-6','n_clicks'),
    Input('4-to-6','n_clicks'),
    [Input(f"6-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    Input('suggest-save-1','n_clicks'),
    Input('suggest-save-2','n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_options_6(*args): # this replaces " add_new_image " 
    print("\n*************************callback: update_dropdown_options_6")
#     print("save button clicked")
    # add image to saved_imgs list and main_dict
    global saved_imgs, all_data, sidebar_outputs, graphs, saved_imgs, suggest_img_px_data, suggest_img_lv_data, suggest_img_cond_data, suggest_img_dimensions, current_cond_data
    
    triggered_id = ctx.triggered_id
    print(f'triggered id: {triggered_id}')
    
    if triggered_id in ['inter-to-6','4-to-6']:
        dropdown_options = update_dropdown_options(saved_imgs)
        return dropdown_options
    
    elif 'delete' in triggered_id:
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])      
            
            dropdown_options = update_dropdown_options(saved_imgs)
            return dropdown_options
        else:
            return no_update
        
    else: # when save button clicked
        next_index = len(all_data['dims'])
        saved_imgs, feedback_msg = update_saved_imgs(next_index,saved_imgs,sidebar_len)
        
        # record which tab the saved img was saved
        saved_imgs_source.insert(0,"6")
        
        # CHRIS EDIT 4---
        lca1 = frameworkCalcs(suggest_img_px_data[0][0,:,:,0])
        equivCost1 = lca1[-1]
        lca2 = frameworkCalcs(suggest_img_px_data[1][0,:,:,0])
        equivCost2 = lca2[-1]
        
        if triggered_id == 'suggest-save-1':
            # use update_all_data function
            all_data['lv_data'] = np.vstack((all_data['lv_data'], suggest_img_lv_data[0]))
            all_data['cond_inputs'] = np.vstack((all_data['cond_inputs'], current_cond_data))
            all_data['px_data'] = np.vstack((all_data['px_data'], suggest_img_px_data[0]))
            all_data['dims'].append(suggest_img_dimensions[0])
            all_data['cost'].append(equivCost1) # CHRIS EDIT

            image_data=255-(all_data['px_data'][-1][:,:,0]*255) # last px_data entry --> added to graphs list for sidebar updates
            graph = dcc.Graph(figure=make_figure(image_data))
            graphs.append(graph)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])

        elif triggered_id == 'suggest-save-2':
            all_data['lv_data'] = np.vstack((all_data['lv_data'], suggest_img_lv_data[1]))
            all_data['cond_inputs'] = np.vstack((all_data['cond_inputs'], current_cond_data))
            all_data['px_data'] = np.vstack((all_data['px_data'], suggest_img_px_data[1]))
            all_data['dims'].append(suggest_img_dimensions[1])
            all_data['cost'].append(equivCost2) # CHRIS EDIT

            image_data=255-(all_data['px_data'][-1][:,:,0]*255) # last px_data entry --> added to graphs list for sidebar updates
            graph = dcc.Graph(figure=make_figure(image_data))
            graphs.append(graph)

            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
        else:
            return no_update 
        
        dropdown_options = update_dropdown_options(saved_imgs)
        return dropdown_options


#TAB 6: UPDATE SIDEBAR AFTER DROPDOWN OPTIONS ARE UPDATED

@app.callback(
    [Output(f'6-saved-image-{j}', 'figure', allow_duplicate = True) for j in range(sidebar_len)],
    [Output(f'6-label-{j}', 'children', allow_duplicate = True) for j in range(sidebar_len)],
    Input("suggest-dropdown","options"),
    prevent_initial_call=True
)
def update_sidebar_6(*args):
    print('\n*************************callback: update_sidebar_6')
    
    return sidebar_outputs

#TAB 6: CLICK IMAGE FROM SIDEBAR & UPDATE DROPDOWN VALUE
    
@app.callback(
    Output('suggest-dropdown','value', allow_duplicate = True),
    [Input(f'6-container-{j}','n_clicks') for j in range(sidebar_len)],
    prevent_initial_call=True
)
def click_sidebar_image_6(*args): # was ' choose_adjust_image '
    print('\n*************************callback: click_sidebar_image_6')
    global saved_imgs
    try:
        clicked_saved_img = int(ctx.triggered_id.split('-')[-1])
    except (ValueError, IndexError):
        print("Empty Saved Image Slot")
        return no_update
    # not sure why it still returns indexerror
    else:
        return saved_imgs[clicked_saved_img] #saved_imgs[len(saved_imgs)-clicked_saved_img-1]
    

# TAB 6: CHOOSE IMAGE 
@app.callback(
    # Output changing lv slider values will trigger callback to update image!
    Output('suggest-image-1','figure'),
    Output('suggest-image-2','figure'),
    Output('suggest-title-1','children'),
    Output('suggest-title-2','children'),
    Output('6-feedback','children'),
    [Input('suggest-button','n_clicks')],
    #[State('suggest-dropdown','value')],
    [Input('suggest-dropdown','value')], # added
    prevent_initial_call=True
)
def suggest_images(n_clicks,value):
    print("\n*************************callback: suggest_images")
    print(ctx.triggered_id)
#     print(f'suggestions based on: #{value}')
    global current_lv_data, current_cond_data, current_image, suggest_img_px_data, suggest_img_lv_data, function_count
    
    #print(f'n_clicks,state_value,value: {n_clicks}, {state_value}, {value}')
    if value == -1:
        return no_update,no_update,no_update,no_update,no_update
    
    num_suggestions=2
    function_count[0] += 1

    # Create several noise_variations
    noise=all_data['lv_data'][value]
    current_lv_data = copy.deepcopy(noise)

    noise.resize((1,latent_dim),refcheck=False)
    noise_variations = adjust_lvs(noise, latent_range, num_suggestions)
    suggest_img_lv_data = noise_variations

    # Generate image variations
    cond_inp = all_data['cond_inputs'][value]
    current_cond_data = copy.deepcopy(cond_inp)
#     suggest_img_cond_data = current_cond_data
    cond_inp.resize((1,latent_dim),refcheck=False)

    feature_imgs = adjust_imgs(noise_variations, cond_inp, num_suggestions, gen_model)

    suggestions = []
    suggest_img_px_data = [] # reset the suggested image data
    costs = [] # CHRIS EDIT
    for img in feature_imgs:
        # adjust dimension to append to all_data
        expanded_img = np.expand_dims(img, axis=0)
        suggest_img_px_data.append(expanded_img)

        img_data = 255-(img*255)
        dimensions = compute_dimensions(img_data) # log dimension data for all_data dict update
        suggest_img_dimensions.append(dimensions)
        # make figures to display
        suggest_fig = make_figure(img_data)
        suggestions.append(suggest_fig)

        # displaying cost
        lca = frameworkCalcs(suggest_img_px_data[-1][0,:,:,0])
        costs.append(lca[-1])

    title1 = f'Image 1: ${costs[0]}'
    title2 = f'Image 2: ${costs[1]}'
    

    """
    print(f'current_lv_data: {current_lv_data}')
    print(f'current_cond_data: {current_cond_data}')
    print(f'current_image: {np.shape(current_image)}')
    plt.imshow(current_image, cmap='gray_r')
    plt.show()
    """

    return suggestions[0], suggestions[1], title1, title2, 'Click the Framework Suggestions button to generate suggestions.'




# ********************* TAB FIN (FINAL) ************************* 
# TAB FIN: OPEN
@app.callback(
    Output('tabs-example','value',allow_duplicate=True),
    [Input(f'{i}-to-fin','n_clicks') for i in ["1","2c0","2c1","2c2","2c3","3","lv","inter","4","5","6"]], # sv edit
    prevent_initial_call=True
)
def display_tab_fin(*args):
    print('\n*************************callback: display_tab_fin')
    global tab_start, tab_end, current_tab, usage_log, function_count, sidebar_outputs
    # log the time spent on the previous tab
    tab_end = time.time() # finish measuring time spent (for last tab)
    time_spent = round(tab_end-tab_start,1)
    log_data = [current_tab,time_spent] + function_count
    usage_log.append(log_data)
    print(log_data)

    # reset the data, so the next callback can record the time spent on this tab
    function_count = [0,0]
    current_tab = "fin"
    tab_start = time.time() # start measuring time spent (for next tab)
    return 'tab-fin'

# TAB FIN: UPDATE DROPDOWN OPTIONS AFTER OPENING TAB 
@app.callback(
    Output("fin-dropdown","options", allow_duplicate=True), 
    Output("fin-feedback", "children", allow_duplicate=True),
    [Input(f'{i}-to-fin','n_clicks') for i in ["1","2c0","2c1","2c2","2c3","3","lv","inter","4","5","6"]],
    [Input(f"fin-delete-{j}", "n_clicks")  for j in range(sidebar_len)],
    prevent_initial_call=True
)
def update_dropdown_options_fin(*args): # this replaces " add_new_image " 
    print("\n*************************callback: update_dropdown_options_fin")

    global saved_imgs, sidebar_outputs
    triggered_id = ctx.triggered_id
    if 'delete' in triggered_id:
        # Handle delete_saved_img trigger
        img_to_del = int(triggered_id.split('-')[-1])
        if len(saved_imgs) != 0:
            saved_imgs.pop(img_to_del)
            saved_imgs_source.pop(img_to_del)
            sidebar_outputs = gen_sidebar_output(saved_imgs, sidebar_len, graphs, all_data["cost"])
            feedback_msg=  'There is no design to delete'
        feedback_msg = f'Design has been deleted'
    else:
        feedback_msg = 'Select your final design'
    dropdown_options = update_dropdown_options(saved_imgs)
  
    return dropdown_options, feedback_msg



#TAB FIN: UPDATE SIDEBAR AFTER DROPDOWN OPTIONS ARE UPDATED

@app.callback(
    [Output(f'fin-saved-image-{j}', 'figure') for j in range(sidebar_len)],
    [Output(f'fin-label-{j}', 'children') for j in range(sidebar_len)],
    [Input(f'{i}-to-fin','n_clicks') for i in ["1","2c0","2c1","2c2","2c3","3","lv","inter","4","5","6"]], # sv edit
    prevent_initial_call=True
)
def update_sidebar_fin(*args):
    print('\n*************************callback: update_sidebar_fin')
    return sidebar_outputs  


#TAB 3: CLICK IMAGE FROM SIDEBAR & UPDATE DROPDOWN VALUE
@app.callback(
    Output('fin-dropdown','value', allow_duplicate=True),
    Output('fin-feedback','children', allow_duplicate=True),
    [Input(f'fin-container-{j}','n_clicks') for j in range(sidebar_len)],
    prevent_initial_call=True
)
def click_sidebar_image_fin(*args): # was ' choose_final_image '
    print('\n*************************callback: click_sidebar_image_fin')
    try:
        clicked_saved_img = int(ctx.triggered_id.split('-')[-1])
        feedback_msg = 'Export your design'
    except (ValueError, IndexError):
        print("Empty Saved Image Slot")
        return no_update, no_update
    # not sure why it still returns indexerror
    else:
        return saved_imgs[clicked_saved_img], feedback_msg #saved_imgs[len(saved_imgs)-clicked_saved_img-1], feedback_msg

    
# TAB FIN: DISPLAY RENDERED IMAGE

@app.callback(
    Output('fin-render','figure',allow_duplicate=True),
    Output('fin-feedback','children',allow_duplicate=True),
    Input('fin-dropdown','value'),
    Input('extrude-button-fin','n_clicks'),
    Input('revolve-button-fin','n_clicks'),
    prevent_initial_call=True
)
def render_3d_view(value,ext,rev):
    print('\ncallback: render_3d_view')
    if ctx.triggered_id == 'revolve-button-fin':
        plot_figure = revolve_3d(all_data['px_data'][value])
    else:
        plot_figure = render_3d(all_data['px_data'][value]) 
    feedback_msg = 'Export your design'

    return plot_figure, feedback_msg


# TAB FIN: EXPORT FINAL
@app.callback(
    #Output('adjust-dropdown',"options",allow_duplicate=True), # dummy
    Output('fin-feedback', 'children', allow_duplicate=True),
    [Input('fin-button-ext','n_clicks'),
     Input('fin-button-rev','n_clicks')],
    State('fin-dropdown','value'),
    prevent_initial_call=True
)
def export_final(click_ext,click_rev,final_choice):
    print("\n*************************callback: export_final")

    final_choice_data = all_data['px_data'][final_choice]
    
    if ctx.triggered_id == 'fin-button-ext':
        create_extrudedSTL(mode,final_choice_data) # export as .stl file to same directory
    elif ctx.triggered_id == 'fin-button-rev':
        create_revolvedSTL(mode,final_choice_data)
    print(usage_log)

    # exporting all data at the end of study
    export_usage_log(mode,usage_log)
    export_saved_img_data(mode,saved_imgs,saved_imgs_source,all_data)
    feedback_msg = 'Your design has been exported!'

    return feedback_msg




# app.run_server(mode="external", host="localhost",port=8055)
app.run_server(mode="inline", host="localhost",port=8052)


function: gen_sidebar_output
saved_imgs: [-1]
function: update_dropdown_options
saved_imgs: [-1]
Dash is running on http://localhost:8052/



CPU times: total: 35.5 s
Wall time: 37.6 s

*************************callback: select_cluster
['1', 536.5, 0, 0]

*************************callback: update_sidebar_c3
trigger: cluster_3_image

*************************callback: update_sidebar_c3
trigger: c3-slot3
function: update_saved_imgs
function: gen_sidebar_output
saved_imgs: [729]

*************************callback: c3_update_eight_images

*************************callback: c3_explore
update images function triggered
8

*************************callback: display_tab_1
['C3', 18.3, 0, 1]

*************************callback: select_cluster
['1', 1.6, 0, 0]

*************************callback: update_sidebar_c2
trigger: cluster_2_image

*************************callback: update_sidebar_c2
trigger: c2-slot3
function: update_saved_imgs
function: gen_sidebar_output
saved_imgs: [490, 729]

*************************callback: display_tab_inter
['C2', 32.0, 0, 0]

*************************callback: display_tab_4
['inter', 1.3, 0, 0]

*******

1/1 [==============================] - 0s 33ms/step
current_lv_data: (12,)
shapes interp imgs, bcs, noise: (13, 128, 96, 1), (13, 12), (13, 12)
input to make_figure is img_data[0]: (128, 96)
img_data (1, 128, 96)

*************************callback: update_sliders
triggered_id: lv_2
current_lv_data: (12,)
current_lv_data: (12,)
function: generate_imgs
1/1 [==============================] - 0s 29ms/step
current_lv_data: (12,)
shapes interp imgs, bcs, noise: (13, 128, 96, 1), (13, 12), (13, 12)
input to make_figure is img_data[0]: (128, 96)
img_data (1, 128, 96)

*************************callback: change_current_img

*************************callback: update_sliders
triggered_id: lv_2
current_lv_data: (12,)
current_lv_data: (12,)
function: generate_imgs
1/1 [==============================] - 0s 29ms/step
current_lv_data: (12,)
shapes interp imgs, bcs, noise: (13, 128, 96, 1), (13, 12), (13, 12)
input to make_figure is img_data[0]: (128, 96)
img_data (1, 128, 96)

*************************

In [ ]:
print(saved_imgs_source)